In [4]:
import numpy as np 
import cv2 as cv
import queue, threading, time 
from configparser import ConfigParser
from numpy.core.numeric import zeros_like
import matplotlib.pyplot as plt
import colorsys
import math
import random
from vidgear.gears import VideoGear
from vidgear.gears import WriteGear
import datetime
import os
import csv

from mpl_toolkits.mplot3d import Axes3D

#hsv transform to rgb format
def hsv2rgb(h,s,v):
    return tuple(round(i * 255) for i in colorsys.hsv_to_rgb(h,s,v))

In [5]:
#control parameters
is_save_stream = False
is_ref_refresh = False
is_anti_shake = False

# default parameters  
# calculation parameters
winsize = 64
mag_ceiling = 0.5
mag_floor = mag_ceiling/5

# camer & image parameters
alpha = 0.8 
hue = 90                     
RES=(1920,1080)                 # camera resolution
# roi_rect = [0,RES[0],0,RES[1]]  # region of interesting
roi_rect = [200,600,115,420] # 转置前

result_RES = (1080, 720)         # flow image resolution
# options = {"CAP_PROP_FRAME_WIDTH":2592, "CAP_PROP_FRAME_HEIGHT":1944, "CAP_PROP_FPS":30}
options = {"CAP_PROP_FRAME_WIDTH":800, "CAP_PROP_FRAME_HEIGHT":600, "CAP_PROP_FPS":30}
# stable calculation parameters
pyr_scale = 0.5
levels = 2
iterations = 1
poly_n = 5
poly_sigma = 1.1
flags = cv.OPTFLOW_FARNEBACK_GAUSSIAN
Noffset = 50 # 消除光流算法偏差的位移量
# drawing buffer
ax = []
ay1 = []
ay2 = []
#plt.ion()
# load parameters from file
# def refresh_parameters():
#     global pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags, mag_min, mag_max, alpha, hue, FPS, BUFFER
#     cfg = ConfigParser()
#     cfg.read("parameters.ini")
#     print("Parameters:")
#     print(cfg.items("Flow"), end="")
#     print(cfg.items("Image"))
#     pyr_scale = float(cfg.get("Flow","pyr_scale"))
#     levels = int(cfg.get("Flow","levels"))
#     winsize = int(cfg.get("Flow","winsize"))
#     iterations = int(cfg.get("Flow","iterations"))
#     poly_n =  int(cfg.get("Flow","poly_n"))
#     poly_sigma = float(cfg.get("Flow","poly_sigma"))
#     flags = int(cfg.get("Flow","flags"))
#     mag_min = float(cfg.get("Image","mag_min"))
#     mag_max = float(cfg.get("Image","mag_max"))
#     alpha = float(cfg.get("Image","alpha"))

# url = r".\video\df1aa122e8a94b784998329ce2b51ed0.mp4"
# url = r"D:\Desktop\BOS-PIV\program\20210731FLOW01\WIN_20220401_12_10_26_Pro.mp4"

In [6]:
# 水平噪声、数值噪声
# stream = VideoGear(source=1, resolution=RES, **options).start()
stream = VideoGear(source=0, stabilize=is_anti_shake, THREADED_QUEUE_MODE=True, **options).start() # To open any valid video stream(for e.g device at 0 index)
# stream = VideoGear(source=0, resolution=RES, stabilize = True).start()

# pre-read frame
frame_pre = stream.read()
print(frame_pre.shape)
frame_pre = frame_pre.transpose(1, 0, 2)
print(frame_pre.shape)
print("video resolution is (height, width, channel) : ",frame_pre.shape)
bgr_pre = cv.cvtColor(frame_pre, cv.COLOR_BGR2GRAY)

# initial hsv
hsv = np.zeros_like(frame_pre[roi_rect[2]:roi_rect[3],roi_rect[0]+Noffset:roi_rect[1]-Noffset])  
# hsv = np.zeros_like(frame_pre[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]])  
hsv[...,1] = 255 #saturation is full
blank = np.zeros_like(frame_pre)

# video loop
count = 0
print("begin stream!")
print("video resolution is (height, width, channel) : ",frame_pre.shape)
# 程序执行区域
timeNow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs("tVariance_" + timeNow)
with open("tVariance_" + timeNow + '/Variance.csv','w',newline='')as csv_file:
    while(True):
    # read current frame from cap
        frame_cur = stream.read()

        bgr_cur = cv.cvtColor(frame_cur, cv.COLOR_BGR2GRAY) # change in to gray
        # # 测量两幅同样图像
        # if is_ref_refresh:
        #     bgr_pre = bgr_cur
        # 图像裁切
        roi_bgr_pre = bgr_pre[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]]
        roi_bgr_cur = bgr_cur[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]]
        # 前后两帧对比
        # if is_ref_refresh:
        #     bgr_pre = bgr_cur

            
        # 图像处理函数：灰度处理、黑白处理
        # # laser处理
        # thresh = 32
        # roi_bgr_cur = ~cv.threshold(roi_bgr_cur, thresh, 255, cv.THRESH_BINARY)[1]
        # roi_bgr_pre = ~cv.threshold(roi_bgr_pre, thresh, 255, cv.THRESH_BINARY)[1]
        # # paper处理
        # thresh = 64
        # roi_bgr_cur = cv.threshold(roi_bgr_cur, thresh, 255, cv.THRESH_BINARY)[1]
        # roi_bgr_pre = cv.threshold(roi_bgr_pre, thresh, 255, cv.THRESH_BINARY)[1]


        # calculate flow
        flow = cv.calcOpticalFlowFarneback(roi_bgr_pre,roi_bgr_cur, None,
                                        pyr_scale,
                                        levels,
                                        winsize,
                                        iterations,
                                        poly_n,
                                        poly_sigma,
                                        flags)
        mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])  # orginal flow
        mag = mag[0:mag.shape[0], Noffset:(mag.shape[1]-Noffset)]
        ang = ang[0:ang.shape[0], Noffset:(ang.shape[1]-Noffset)]

        mag_mean = cv.mean(mag)[0]
        mag_sft = abs(mag - mag_mean)  # shifted magnitude to elimiate noise
        hsv[...,0] = (ang + hue)*180/np.pi/2 # color space related to angle 
        hsv[...,2] = cv.normalize(mag_sft,None,0,255,cv.NORM_MINMAX) 
        
        #enhanced flow
        bgr_flow = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
        mag_enhanced = zeros_like(mag_sft)
        cv.min(mag_sft,mag_ceiling,mag_enhanced)  # enhance flow, ceiling and flooring
        cv.max(mag_enhanced,mag_floor,mag_enhanced)
        hsv[...,0] = (ang + hue)*180/np.pi/2 # color space related to angle 
        hsv[...,2] = cv.normalize(mag_enhanced,None,0,255,cv.NORM_MINMAX)
        bgr_flow_enhanced = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
        
        # image emerge with enhanced flow
        flow_blend_enhance = cv.addWeighted(frame_cur[roi_rect[2]:roi_rect[3],roi_rect[0]+Noffset:roi_rect[1]-Noffset], 1-alpha ,bgr_flow_enhanced,  alpha, 0)
        bgr_blend = cv.addWeighted(frame_cur[roi_rect[2]:roi_rect[3],roi_rect[0]+Noffset:roi_rect[1]-Noffset], 1-alpha ,bgr_flow,  alpha, 0)
        bgr_blend = bgr_blend[0:bgr_blend.shape[0], Noffset:bgr_blend.shape[0]-Noffset]
        frame_blend = cv.addWeighted(frame_cur, 1-alpha ,blank,  alpha, 0)#frame_cur.copy()
        frame_blend[roi_rect[2]:roi_rect[3],roi_rect[0]+Noffset:roi_rect[1]-Noffset] = flow_blend_enhance

        #  # image emerge with enhanced flow
        # flow_blend_enhance = cv.addWeighted(frame_cur[roi_rect[2]:roi_rect[3],roi_rect[0]-Noffset:roi_rect[1]], 1-alpha ,bgr_flow_enhanced,  alpha, 0)
        # bgr_blend = cv.addWeighted(frame_cur[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]], 1-alpha ,bgr_flow,  alpha, 0)
        # bgr_blend = bgr_blend[0:bgr_blend.shape[0], Noffset:bgr_blend.shape[0]-Noffset]
        # frame_blend = cv.addWeighted(frame_cur, 1-alpha ,blank,  alpha, 0)#frame_cur.copy()
        # frame_blend[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]] = flow_blend_enhance

        # if ture, compare image is  successive; if false, compare image is not change
        if is_ref_refresh:
            bgr_pre = bgr_cur
        
        #palette
        palette=np.zeros((512,512,3),np.uint8)
        mag_norm = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
        mag_min,mag_max,min_indx,max_indx=cv.minMaxLoc(mag)
        ang_mean = cv.mean(ang*mag/mag_mean)[0]

        # 计算水平噪声最大值和竖直噪声最大值
        _,hor_MaxNoise, _, _ = cv.minMaxLoc(np.multiply(mag, np.cos(ang)))
        _,ver_MaxNoise, _, _ = cv.minMaxLoc(np.multiply(mag, np.sin(ang)))


        # 计算水平噪声和竖直噪声
        hor_Noise = np.average(np.multiply(mag, np.cos(ang)))
        ver_Noise = np.average(np.multiply(mag, np.sin(ang)))
        # 绝对值计算水平噪声和竖直噪声
        hor_AbsNoise = np.average(np.abs(np.multiply(mag, np.cos(ang))))
        ver_AbsNoise = np.average(np.abs(np.multiply(mag, np.sin(ang))))


        cv.putText(palette, "max="+str(mag.max()), (0, 30), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "mag_mean="+str("%.7f"%mag_mean), (0, 60), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "mag_max="+str(mag_max), (0, 90), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "mag_min="+str(mag_min), (0, 120), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "hor_Noise="+str(hor_Noise), (0, 150), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "ver_Noise="+str(ver_Noise), (0, 180), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "hor_MaxNoise="+str(hor_MaxNoise), (0, 210), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
        cv.putText(palette, "ver_MaxNoise="+str(ver_MaxNoise), (0, 240), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)

        timeNow1 = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        # max min min variance 
        # 获取一个csv对象进行内容写入
        writer=csv.writer(csv_file)
        # writerow 写入一行数据
        writeDatum = [timeNow1, mag_mean, mag_max, mag_min, hor_Noise, ver_Noise, hor_MaxNoise, ver_MaxNoise]
        writer.writerow(writeDatum)

        count = count + 1
    
        # result flow image
        cv.imshow('Orignal',cv.resize(frame_cur, (result_RES[0], result_RES[1])))
        cv.imshow("Palette", palette)
        cv.imshow('Blender', cv.resize(bgr_blend , (result_RES[0], result_RES[1])))
        cv.imshow('Area of Intrest Blended', cv.resize(frame_blend  , (result_RES[0], result_RES[1])))
        
        key = cv.waitKey(10)
        if key==114: # if input key 'r', refresh compare image
            #打印时间戳保存
            timeNow1 = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            os.makedirs("Refresh_" + timeNow1)
            bgr_pre = bgr_cur
            # cv.imwrite(timeNow +'/Orignal.jpg',frame_cur)
            # cv.imwrite(timeNow +'/Area_of_Intrest_Blended.jpg', frame_blend)
            cv.imwrite(timeNow1 +"/Palette.jpg", palette)
            cv.imwrite(timeNow1 +'/Blender.jpg', bgr_blend)
            print("the image is refreshed")

(600, 800, 3)
(800, 600, 3)
video resolution is (height, width, channel) :  (800, 600, 3)
begin stream!
video resolution is (height, width, channel) :  (800, 600, 3)


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-c2l3r8zm\opencv\modules\core\src\arithm.cpp:650: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'


In [ ]:
# stream = VideoGear(source=1, resolution=RES, **options).start()
stream = VideoGear(source=0, stabilize=is_anti_shake, THREADED_QUEUE_MODE=True, **options).start() # To open any valid video stream(for e.g device at 0 index)
# stream = VideoGear(source=0, resolution=RES, stabilize = True).start()

# pre-read frame
frame_pre = stream.read()
print("video resolution is (height, width, channel) : ",frame_pre.shape)
bgr_pre = cv.cvtColor(frame_pre, cv.COLOR_BGR2GRAY)

# initial hsv
hsv = np.zeros_like(frame_pre[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]])  
hsv[...,1] = 255 #saturation is full
blank = np.zeros_like(frame_pre)

# video loop
count = 0
print("begin stream!")
print("video resolution is (height, width, channel) : ",frame_pre.shape)

while(True):
    # read current frame from cap
        frame_cur = stream.read()
        bgr_cur = cv.cvtColor(frame_cur, cv.COLOR_BGR2GRAY) # change in to gray
        
        # 图像裁切
        roi_bgr_cur = bgr_cur[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]]
    

        # 图像处理函数：灰度处理、黑白处理
        thresh = 16
        roi_bgr_cur = cv.threshold(roi_bgr_cur, thresh, 255, cv.THRESH_BINARY)[1]

        cv.imshow('Orignal',cv.resize(~roi_bgr_cur, (result_RES[0], result_RES[1])))
        key = cv.waitKey(10)

video resolution is (height, width, channel) :  (600, 800, 3)
begin stream!
video resolution is (height, width, channel) :  (600, 800, 3)


KeyboardInterrupt: 